In [ ]:
import pulp
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Tworzenie funkcji optymalizacyjnej
def solve_problem(czekolada_price, karmel_price, olej_price, orzechy_price, cukier_price, zasoby_czekolada, zasoby_orzechy):
    # Problem optymalizacji
    problem = pulp.LpProblem("Optymalizacja_produkcji_batonow", pulp.LpMaximize)
    
    # Zmienne decyzyjne
    mars = pulp.LpVariable('Mars', lowBound=0, cat='Integer')
    snickers = pulp.LpVariable('Snickers', lowBound=0, cat='Integer')
    
    # Koszt składników za baton
    mars_cost = 0.5 * czekolada_price + 0.2 * karmel_price + 0.3 * orzechy_price + 0.05 * olej_price
    snickers_cost = 0.4 * czekolada_price + 0.2 * karmel_price + 0.3 * cukier_price + 0.05 * olej_price + 0.2 * orzechy_price
    
    # Maksymalizacja zysku
    problem += 100 * mars + 105 * snickers, "Całkowity zysk"
    
    # Ograniczenia zasobów
    problem += 0.5 * mars + 0.4 * snickers <= zasoby_czekolada, "Zasoby_czekolady"
    problem += 0.2 * snickers <= zasoby_orzechy, "Zasoby_orzechow"
    
    # Rozwiązanie problemu
    problem.solve()
    
    # Wyniki
    mars_value = mars.varValue
    snickers_value = snickers.varValue
    revenue = pulp.value(problem.objective)
    profit = revenue - (mars_value * mars_cost + snickers_value * snickers_cost)
    
    return mars_value, snickers_value, revenue, profit

# Funkcja wizualizacyjna
def plot_production(zasoby_czekolada, zasoby_orzechy):
    orzechy_range = range(0, int(zasoby_orzechy) + 1, 100)
    mars_values = []
    snickers_values = []
    
    for orzechy_limit in orzechy_range:
        mars, snickers, _, _ = solve_problem(
            czekolada_slider.value, karmel_slider.value, olej_slider.value,
            orzechy_slider.value, cukier_slider.value, zasoby_czekolada, orzechy_limit
        )
        mars_values.append(mars)
        snickers_values.append(snickers)
    
    plt.figure(figsize=(10, 6))
    plt.plot(orzechy_range, mars_values, label="Mars (kg)", color='blue')
    plt.plot(orzechy_range, snickers_values, label="Snickers (kg)", color='orange')
    plt.xlabel("Dostępne zasoby orzechów (kg)")
    plt.ylabel("Produkcja (kg)")
    plt.title("Wpływ zasobów orzechów na produkcję batonów")
    plt.legend()
    plt.grid(True)
    plt.show()

# Slidery
czekolada_slider = widgets.FloatSlider(value=20, min=10, max=30, step=1, description='Cena czekolady')
karmel_slider = widgets.FloatSlider(value=15, min=10, max=20, step=1, description='Cena karmelu')
olej_slider = widgets.FloatSlider(value=7, min=5, max=15, step=1, description='Cena oleju')
orzechy_slider = widgets.FloatSlider(value=20, min=10, max=30, step=1, description='Cena orzechów')
cukier_slider = widgets.FloatSlider(value=4, min=2, max=10, step=1, description='Cena cukru')
zasoby_czekolada_slider = widgets.IntSlider(value=6000, min=1000, max=10000, step=500, description='Zasoby czekolady')
zasoby_orzechy_slider = widgets.IntSlider(value=2000, min=500, max=5000, step=100, description='Zasoby orzechów')

# Kontener na wyjście wyników i wykresów
output = widgets.Output()

def update_production(change=None):
    with output:
        clear_output(wait=True)  # Clear only the output area
        # Obliczenia zaktualizowane na podstawie wartości suwaków
        mars, snickers, revenue, profit = solve_problem(
            czekolada_slider.value, karmel_slider.value, olej_slider.value,
            orzechy_slider.value, cukier_slider.value,
            zasoby_czekolada_slider.value, zasoby_orzechy_slider.value
        )
        
        # Wyświetlenie wyników
        print(f"Produkuj {mars:.0f} kg Marsów dziennie")
        print(f"Produkuj {snickers:.0f} kg Snickersów dziennie")
        print(f"Całkowity przychód: {revenue:.2f} zł")
        print(f"Całkowity zysk: {profit:.2f} zł")
        
        # Aktualizacja wykresu
        plot_production(zasoby_czekolada_slider.value, zasoby_orzechy_slider.value)

# Połączenie widgetów z funkcją
czekolada_slider.observe(update_production, names='value')
karmel_slider.observe(update_production, names='value')
olej_slider.observe(update_production, names='value')
orzechy_slider.observe(update_production, names='value')
cukier_slider.observe(update_production, names='value')
zasoby_czekolada_slider.observe(update_production, names='value')
zasoby_orzechy_slider.observe(update_production, names='value')

# Wyświetlenie widgetów i wyników
display(widgets.VBox([
    czekolada_slider, karmel_slider, olej_slider, orzechy_slider, cukier_slider,
    zasoby_czekolada_slider, zasoby_orzechy_slider, output
]))

# Początkowe obliczenia i wykres
update_production()
